In [28]:
import numpy as np 
import pandas as pd 
import plotly.express as px  # (version 4.7.0)
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio 
from datetime import datetime
pio.renderers.default='browser'
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [15]:
def create_dataframe(covid_file,dep_pop_file,dep_sup_file):
    dep_sup = pd.read_csv(dep_sup_file,dtype={'dep': str, 'code': str, 'Superficie(km2)': float})
    dep_pop = pd.read_csv(dep_pop_file,dtype={'dep': str, 'code': str, 'Population(milliers)': float})
    covid = pd.read_csv(covid_file)
    dep_sup = dep_sup.sort_values(by=['code'])
    dep_pop['Population(milliers)'] = dep_pop['Population(milliers)'] * 1000
    dep_sup['Population(hab)'] = (dep_pop['Population(milliers)'].values)
    dep_sup['Density(hab/km2)'] =round(dep_sup['Population(hab)']/dep_sup['Superficie(km2)'],2)
    #dep_sup['Density(hab/km2)_log'] = np.log10(dep_sup['Density(hab/km2)'])
    print(dep_sup)
    dep_sup.to_csv('france_density_pop.csv',index=False)
    
    #return covid


In [37]:
    def create_df(covid_file,franced):
        covid = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7', sep=";", parse_dates=True)
        density = pd.read_csv(franced,dtype={'name_dep': str, 'dep': str, 'Superficie(km2)': float,'Population(milliers)': float,'Density(hab/km2)':float})
        covid = pd.merge(density, covid, on = ['dep'])
        for i in ['974','973','972','971']: 
            covid= covid.drop(covid[ covid['dep'] ==i].index, axis=0)
        covid['sum']= covid['hosp'] + covid['rea']
        covid['sum']=covid['sum'].astype('float64')
        np.seterr(divide = 'ignore')
        for i in range(len(covid['sum'])): 
            if np.log(covid['sum'][i])==float('-inf'):
                covid['sum'].loc[i]=1
        np.seterr(divide = 'warn')
        print(covid)
        covid['jour'] =pd.to_datetime(covid['jour']).dt.date
        covid = covid.set_index('jour')
        return covid

In [38]:
covid = create_df("Data/donnees-hospitalieres-covid.csv","Data/France_density_pop.csv")

         name_dep dep  Superficie(km2)  Population(hab)  Density(hab/km2)  \
0             Ain  01           5762.0         657000.0            114.02   
1             Ain  01           5762.0         657000.0            114.02   
2             Ain  01           5762.0         657000.0            114.02   
3             Ain  01           5762.0         657000.0            114.02   
4             Ain  01           5762.0         657000.0            114.02   
...           ...  ..              ...              ...               ...   
60187  Val-d’Oise  95           1246.0        1248400.0           1001.93   
60188  Val-d’Oise  95           1246.0        1248400.0           1001.93   
60189  Val-d’Oise  95           1246.0        1248400.0           1001.93   
60190  Val-d’Oise  95           1246.0        1248400.0           1001.93   
60191  Val-d’Oise  95           1246.0        1248400.0           1001.93   

       sexe        jour  hosp  rea   rad   dc    sum  
0         0  2020-03

In [39]:
covid

,name_dep,dep,Superficie(km2),Population(hab),Density(hab/km2),sexe,hosp,rea,rad,dc,sum
jour,,,,,,,,,,,
2020-03-18,Ain,01,5762.0,657000.0,114.02,0,2,0,1,0,2.0
2020-03-18,Ain,01,5762.0,657000.0,114.02,1,1,0,1,0,1.0
2020-03-18,Ain,01,5762.0,657000.0,114.02,2,1,0,0,0,1.0
2020-03-19,Ain,01,5762.0,657000.0,114.02,0,2,0,1,0,2.0
2020-03-19,Ain,01,5762.0,657000.0,114.02,1,1,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2020-10-11,Val-d’Oise,95,1246.0,1248400.0,1001.93,1,103,19,1837,493,122.0
2020-10-11,Val-d’Oise,95,1246.0,1248400.0,1001.93,2,86,14,1446,296,100.0
2020-10-12,Val-d’Oise,95,1246.0,1248400.0,1001.93,0,198,36,3300,791,234.0


In [16]:
covid = create_dataframe("Data/donnees-hospitalieres-covid.csv","Data/dep-pop.csv","Data/dep-sup.csv")
#france = json.load(open("Geojson/france_dep.geojson",'r'))

                        dep code  Superficie(km2)  Population(hab)  \
56                      Ain   01           5762.0         657000.0   
12                    Aisne   02           7369.0         526100.0   
13                   Allier   03           7340.0         331300.0   
17  Alpes-de-Haute-Provence   04           6925.0         165200.0   
61             Hautes-Alpes   05           5549.0         141800.0   
..                      ...  ...              ...              ...   
92               Val-d’Oise   95           1246.0        1248400.0   
91               Guadeloupe  971           1703.0         376900.0   
93               Martinique  972           1128.0         358700.0   
0                    Guyane  973          83534.0         290700.0   
88                  Réunion  974           2504.0         860000.0   

    Density(hab/km2)  
56            114.02  
12             71.39  
13             45.14  
17             23.86  
61             25.55  
..               ... 

In [8]:
covid

,dep,code,Superficie(km2),Population(hab),Density(hab/km2),Density(hab/km2)_log,sexe,hosp,rea,rad,dc,sum
jour,,,,,,,,,,,,
2020-03-18,Ain,01,5762.0,657000.0,114.02,2.056981,0,2,0,1,0,2.0
2020-03-18,Ain,01,5762.0,657000.0,114.02,2.056981,1,1,0,1,0,1.0
2020-03-18,Ain,01,5762.0,657000.0,114.02,2.056981,2,1,0,0,0,1.0
2020-03-19,Ain,01,5762.0,657000.0,114.02,2.056981,0,2,0,1,0,2.0
2020-03-19,Ain,01,5762.0,657000.0,114.02,2.056981,1,1,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-07,Val-d’Oise,95,1246.0,1248400.0,1001.93,3.000837,1,106,21,1813,489,127.0
2020-10-07,Val-d’Oise,95,1246.0,1248400.0,1001.93,3.000837,2,83,11,1434,294,94.0
2020-10-08,Val-d’Oise,95,1246.0,1248400.0,1001.93,3.000837,0,191,33,3266,786,224.0


In [190]:
d = {int(i):str(j) for i,j in zip(range(len(covid.index.unique())), covid.index.unique())}

In [213]:
d2

{14: 'Avril',
 44: 'Mai',
 75: 'Juin',
 105: 'Juillet',
 136: 'Aout',
 167: 'Septembre',
 197: 'Octrobre'}

In [202]:
mois = {'01':'Janvier','02':'Février','03':'Mars','04':'Avril','05':'Mai','06':'Juin','07':'Juillet','08':'Aout','09':'Septembre','10':'Octrobre','11':'Novembre','12':'Decembre'}
def transform(date): 
    tmp = date.split('-')
    s = tmp[2] +' '+mois[tmp[1]]+' '+tmp[0]
    return s 

transform(d[24])

11 Avril 2020


In [306]:
# get specific data for 1 sexe dans 1 day
covid = covid[covid['sexe']==2]
covid = covid.loc[datetime.strptime(d[0], '%Y-%m-%d').date()]
covid = covid.sort_values(by=['Density(hab/km2)'])

In [300]:
covid[covid['sum']==0]

,dep,code,Superficie(km2),Population(hab),Density(hab/km2),Density(hab/km2)_log,sexe,hosp,rea,rad,dc,sum
jour,,,,,,,,,,,,


In [307]:
#ig = go.Figure()

fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.35, 0.65],
    horizontal_spacing=0.03,
    specs=[[{"type": "Choroplethmapbox"}, {"type": "bar"}]],
    subplot_titles=("Hospitalisation et Réanimation par département en France","Hospitalisation et Réanimation par département en France en fonction de la densité"))

fig.add_trace(go.Bar(
    x=covid['dep'],
    y=covid['hosp'],
    name='Hospitalisation',
    hovertemplate ="<b> Departement : %{customdata[0]} </b><br><b> Densité : %{customdata[4]} hab/km2</b><br><b>Hospitalisation : %{customdata[7]} pers.</b>"+"<extra></extra>",
    customdata=covid,
    marker_color='rgb(196, 102, 73)',
    showlegend =False),
    row=1, col=2)

fig.add_trace(go.Bar(
    x=covid['dep'],
    y=covid['rea'],
    name='Réanimation',
    hovertemplate ="<b> Departement : %{customdata[0]} </b><br><b> Densité : %{customdata[4]} hab/km2</b><br><b>Réanimation : %{customdata[8]} pers.</b>"+"<extra></extra>",
    customdata=covid,
    marker_color='rgb(89, 13, 31)',
    showlegend =False),
    row=1, col=2)

fig.add_trace(go.Choroplethmapbox(geojson=france, 
                                    locations=covid['code'],
                                    z=np.log10(covid['sum']),
                                    customdata=covid,
                                    showscale=False,
                                    hovertemplate ="<b> Departement : %{customdata[0]} </b><br><b> Densité : %{customdata[4]} hab/km2</b><br><b>Hospitalisation : %{customdata[7]} pers.</b><br><b>Réanimation : %{customdata[8]} pers.</b>"+"<extra></extra>",
                                    featureidkey="properties.code",                           
                                    colorscale="amp",
                                    showlegend = False,
                                    marker_opacity=0.8, 
                                    marker_line_width=0.3,
                                    name='europe_map'),
                                    row=1, col=1)

fig.update_layout(title = go.layout.Title(text = 'Density Urbaine et covid'),mapbox_style="carto-positron",mapbox_zoom=4.7, mapbox_center = {"lat": 46.35, "lon": 2.55})

fig.update_layout(barmode='relative')
fig.update_layout(template='plotly_white')

fig.show()